In [11]:
# Machine learning stratification

In [12]:
# Validación cruzada (mediante muestreo aleatorio repetido) para evaluar la estabilidad de los agrupamientos.
# Realiza estratificación con 3, 4 y 5 estratos.
# Compara cuatro representaciones: Original, PCA, Kernel PCA, Autoencoder.

In [13]:
# Importar librerias necesarias
import pandas as pd
import numpy as np
import time
from itertools import product
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, KernelPCA
from sklearn.cluster import KMeans, MiniBatchKMeans, AgglomerativeClustering, Birch, SpectralClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import friedmanchisquare
import random
import tensorflow as tf
from google.colab import files

In [14]:
# Instalar la libreria que permite leer archivos en formato .rds
!pip install pyreadr

In [15]:
from google.colab import files
uploaded = files.upload()

Saving 9_PSU_indicators_Cuenca.rds to 9_PSU_indicators_Cuenca (1).rds


In [16]:
# Dataset de las UPM de Cuenca
import pyreadr
# Leer archivo .rds
result = pyreadr.read_r('9_PSU_indicators_Cuenca.rds')
# Extraer el dataframe (puede haber más de uno)
df = result[None]  # usualmente el primero está en None
# Ver los datos
df.head()

,id_upm,area,dom_area,p_educ_superior,p_pers_ocupad,p_homb_ocupad,p_muje_ocupad,p_no_ocupad_15_17,p_pers_no_desocupad,p_homb_no_desocupad,...,v_dispone_luz,h_serv_higienico_es,h_lugar_cocinar,h_dispone_ducha,h_dispone_telef,h_dispone_celular,h_dispone_internet,h_dispone_compu,h_dispone_tvcable,h_todos_bienes
0,010150001001,1,271,23.32,61.67,72.73,52.26,82.35,94.65,92.31,...,100.00,82.61,95.65,79.13,32.17,84.35,7.83,45.22,0.00,0.00
1,010150001002,1,271,18.18,66.32,77.78,56.31,67.57,93.77,93.96,...,100.00,71.34,85.99,69.43,14.01,69.43,11.46,35.67,1.27,0.00
2,010150001003,1,271,33.53,67.04,79.55,54.81,72.00,98.35,98.13,...,100.00,91.30,93.48,83.70,27.17,59.78,8.70,38.04,3.26,2.17
3,010150001004,1,271,14.71,64.74,78.72,53.22,79.07,99.02,99.11,...,99.15,78.99,94.12,74.79,29.41,81.51,10.08,43.70,2.52,0.84
4,010150001005,1,271,23.90,60.57,72.22,50.87,93.02,95.05,94.55,...,99.12,90.52,89.66,82.76,41.38,63.79,4.31,46.55,3.45,0.00


In [17]:
# Escalar los datos
df_sel = df.iloc[:, 3:43]
X = StandardScaler().fit_transform(df_sel)
#X = X[0:1000]
X = X.astype(np.float32)
print(X)
print(X.shape)

[[-0.7375582  -0.28730026 -0.18778428 ... -0.6436218  -1.241332
  -0.81344384]
 [-1.0943717   0.82130396  0.77188015 ... -1.2772564  -1.1403009
  -0.81344384]
 [-0.02879065  0.9929588   1.1082377  ... -1.1200088  -0.9819921
  -0.5355305 ]
 ...
 [-1.9975122   0.2681939   0.35380852 ... -1.5287198  -0.8952803
  -0.81344384]
 [-1.3935673   1.7105714   2.6722057  ... -2.148421   -1.241332
  -0.81344384]
 [-1.7934206  -1.0692835   0.20368278 ... -1.5871071  -1.0822278
  -0.6853732 ]]
(781, 40)


In [18]:
# Definicion de funciones
# ------------------------------- CONFIGURACION -------------------------------

config_dict = {
    "representations": [
        {"name": "Original"},
        {"name": "PCA", "n_components": [2, 3, 5]},
        {"name": "Kernel PCA", "n_components": [2], "kernel": ["rbf", "poly"], "gamma": [0.1, 1]},
        {"name": "Autoencoder", "encoding_dim": [2, 3], "layers": [(64,), (128, 64)], "dropout": [0.0, 0.2], "epochs": [50], "batch_size": [32]}
    ],
    "clustering": [
        {"name": "KMeans", "n_clusters": [3, 4, 5], "n_init": [10, 20]},
        {"name": "MiniBatchKMeans", "n_clusters": [3, 4, 5], "batch_size": [50, 100]},
        {"name": "Agglomerative", "n_clusters": [3, 4, 5], "linkage": ["ward", "average"]},
        {"name": "BIRCH", "n_clusters": [3, 4, 5], "threshold": [0.5, 1.0]},
        {"name": "GaussianMixture", "n_components": [3, 4, 5], "covariance_type": ["full", "diag"]},
        {"name": "SpectralClustering", "n_clusters": [3, 4, 5], "gamma": [0.5, 1.0]}
    ]
}

# -------------------------- FUNCIONES DE TRANSFORMACION ----------------------

def autoencoder_custom(X, encoding_dim=2, layers=(64,), dropout=0.0, epochs=50, batch_size=32):
    input_dim = X.shape[1]
    input_layer = Input(shape=(input_dim,))
    x = input_layer
    for units in layers:
        x = Dense(units, activation='relu')(x)
        if dropout > 0.0:
            x = Dropout(dropout)(x)
    encoded = Dense(encoding_dim, activation='relu')(x)
    decoded = Dense(input_dim, activation='linear')(encoded)
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)
    autoencoder.compile(optimizer=Adam(), loss='mse')
    autoencoder.fit(X, X, epochs=epochs, batch_size=batch_size, verbose=0)
    return encoder, encoder.predict(X)

def get_representations(X, rep_conf):
    name = rep_conf["name"]
    if name == "Original":
        return {"Original": (X, {}, lambda X_new: X_new)}
    elif name == "PCA":
        valid_components = [n for n in rep_conf["n_components"] if n <= X.shape[1]]
        return {
            f"PCA (n={n})": (PCA(n_components=n).fit(X).transform(X), {"n_components": n}, PCA(n_components=n).fit(X).transform)
            for n in valid_components
        }
    elif name == "Kernel PCA":
        valid_components = [n for n in rep_conf["n_components"] if n <= X.shape[1]]
        return {
            f"KernelPCA (kernel={k}, gamma={g})": (
                KernelPCA(n_components=n, kernel=k, gamma=g).fit(X).transform(X),
                {"n_components": n, "kernel": k, "gamma": g},
                KernelPCA(n_components=n, kernel=k, gamma=g).fit(X).transform
            )
            for n in valid_components for k in rep_conf["kernel"] for g in rep_conf["gamma"]
        }
    elif name == "Autoencoder":
        result = {}
        for d in rep_conf["encoding_dim"]:
            for layers in rep_conf["layers"]:
                for dropout in rep_conf["dropout"]:
                    encoder, X_encoded = autoencoder_custom(X, encoding_dim=d, layers=layers, dropout=dropout,
                                                             epochs=rep_conf["epochs"][0], batch_size=rep_conf["batch_size"][0])
                    result[f"Autoencoder (dim={d}, layers={layers}, dropout={dropout})"] = (
                        X_encoded,
                        {"encoding_dim": d, "layers": layers, "dropout": dropout, "epochs": rep_conf["epochs"][0], "batch_size": rep_conf["batch_size"][0]},
                        encoder.predict
                    )
        return result
    return {}

# -------------------------- FUNCIONES DE CLUSTERING ---------------------------

def get_clustering_methods(clust_conf):
    methods = []
    name = clust_conf["name"]
    if name == "KMeans":
        for k, n_init in product(clust_conf["n_clusters"], clust_conf["n_init"]):
            methods.append((f"KMeans (k={k})", lambda X, k=k, n_init=n_init: KMeans(n_clusters=k, n_init=n_init, random_state=42).fit_predict(X), {"n_clusters": k, "n_init": n_init}))
    elif name == "MiniBatchKMeans":
        for k, b in product(clust_conf["n_clusters"], clust_conf["batch_size"]):
            methods.append((f"MiniBatchKMeans (k={k})", lambda X, k=k, b=b: MiniBatchKMeans(n_clusters=k, batch_size=b, random_state=42).fit_predict(X), {"n_clusters": k, "batch_size": b}))
    elif name == "Agglomerative":
        for k, linkage in product(clust_conf["n_clusters"], clust_conf["linkage"]):
            methods.append((f"Agglomerative (k={k}, linkage={linkage})", lambda X, k=k, linkage=linkage: AgglomerativeClustering(n_clusters=k, linkage=linkage).fit_predict(X), {"n_clusters": k, "linkage": linkage}))
    elif name == "BIRCH":
        for k, t in product(clust_conf["n_clusters"], clust_conf["threshold"]):
            methods.append((f"BIRCH (k={k}, t={t})", lambda X, k=k, t=t: Birch(n_clusters=k, threshold=t).fit_predict(X), {"n_clusters": k, "threshold": t}))
    elif name == "GaussianMixture":
        for k, cov in product(clust_conf["n_components"], clust_conf["covariance_type"]):
            methods.append((f"GMM (k={k}, cov={cov})", lambda X, k=k, cov=cov: GaussianMixture(n_components=k, covariance_type=cov, random_state=42).fit(X).predict(X), {"n_components": k, "covariance_type": cov}))
    elif name == "SpectralClustering":
        for k, g in product(clust_conf["n_clusters"], clust_conf["gamma"]):
            methods.append((f"Spectral (k={k}, gamma={g})", lambda X, k=k, g=g: SpectralClustering(n_clusters=k, affinity='rbf', gamma=g, assign_labels='kmeans', random_state=42).fit_predict(X), {"n_clusters": k, "gamma": g}))
    return methods

# ------------------------- FUNCIONES DE EJECUCIÓN FINAL -----------------------

def run_experiments(X):
    all_results = []
    label_results = {}
    label_columns = {}
    best_results_by_method = []

    cv = KFold(n_splits=5, shuffle=True, random_state=42)

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X)):
        X_fold = X[train_idx]
        for rep_conf in config_dict['representations']:
            representations = get_representations(X_fold, rep_conf)
            for rep_name, (X_rep, rep_params, transformer) in representations.items():
                X_test_rep = transformer(X[test_idx])
                for clust_conf in config_dict['clustering']:
                    methods = get_clustering_methods(clust_conf)
                    for method_name, cluster_func, clust_params in methods:
                        start = time.time()
                        try:
                            labels = cluster_func(X_rep)
                            elapsed = time.time() - start

                            if len(set(labels)) < 2:
                                raise ValueError("Se generó solo un clúster. No se puede calcular métricas de calidad.")

                            silhouette = silhouette_score(X_rep, labels)
                            davies = davies_bouldin_score(X_rep, labels)

                            all_results.append({
                                'Representación': rep_name,
                                'Método': method_name,
                                'Tiempo (s)': elapsed,
                                'Silhouette': silhouette,
                                'Davies-Bouldin': davies,
                                'Params_Representación': rep_params,
                                'Params_Clustering': clust_params
                            })

                            key = f"{rep_name} + {method_name}"
                            if key not in label_columns:
                                label_columns[key] = np.full(len(X), np.nan)
                            label_columns[key][test_idx] = cluster_func(X_test_rep)

                            label_results[(rep_name, method_name, str(start))] = labels

                        except Exception as e:
                            elapsed = time.time() - start
                            print(f"Error en {rep_name} + {method_name}: {e}")
                            all_results.append({
                                'Representación': rep_name,
                                'Método': method_name,
                                'Tiempo (s)': elapsed,
                                'Silhouette': None,
                                'Davies-Bouldin': None,
                                'Error': str(e),
                                'Params_Representación': rep_params,
                                'Params_Clustering': clust_params
                            })

    final_labels = pd.DataFrame(label_columns)
    final_labels.to_csv("etiquetas_finales_cross_validation.csv")

    df = pd.DataFrame(all_results)
    df.to_csv("todos_los_resultados.csv", index=False)

    df_validos = df.dropna(subset=['Silhouette', 'Davies-Bouldin'])
    resumen_metricas = df_validos.groupby(['Representación', 'Método']).agg({
        'Silhouette': ['mean', 'std', lambda x: np.std(x) / np.mean(x) if np.mean(x) != 0 else 0],
        'Davies-Bouldin': ['mean', 'std', lambda x: np.std(x) / np.mean(x) if np.mean(x) != 0 else 0],
        'Tiempo (s)': ['mean', 'std']
    }).reset_index()
    resumen_metricas.columns = ['Representación', 'Método', 'Silhouette_Media', 'Silhouette_STD', 'Silhouette_CV',
                                'DB_Media', 'DB_STD', 'DB_CV', 'Tiempo_Medio', 'Tiempo_STD']

    # ------------------- Score normalizado MinMax (Global y por Método) ---------------------
    def min_max_normalize(series):
        return (series - series.min()) / (series.max() - series.min())

    resumen_metricas['Silhouette_Norm'] = min_max_normalize(resumen_metricas['Silhouette_Media'])
    resumen_metricas['DB_Norm'] = min_max_normalize(resumen_metricas['DB_Media'])
    resumen_metricas['Score_Norm_Global'] = resumen_metricas['Silhouette_Norm'] - resumen_metricas['DB_Norm']

    resumen_metricas.to_csv("resumen_metricas_clustering.csv", index=False)

    # Score normalizado por Método
    score_por_metodo = []
    for method in resumen_metricas['Método'].unique():
        sub = resumen_metricas[resumen_metricas['Método'] == method].copy()
        sub['Silhouette_Norm'] = min_max_normalize(sub['Silhouette_Media'])
        sub['DB_Norm'] = min_max_normalize(sub['DB_Media'])
        sub['Score_Norm_Por_Metodo'] = sub['Silhouette_Norm'] - sub['DB_Norm']
        score_por_metodo.append(sub)

    score_metodo_df = pd.concat(score_por_metodo, ignore_index=True)
    score_metodo_df.to_csv("score_normalizado_por_metodo.csv", index=False)

    # ------------------- Añadir Prueba de Friedman ---------------------
    df_validos = resumen_metricas.copy()
    df_ranks = df_validos.groupby(['Representación', 'Método'])[['Score_Norm_Global']].mean().reset_index()
    matriz_rendimiento = df_ranks.pivot(index='Representación', columns='Método', values='Score_Norm_Global').dropna(axis=1)
    ranking = matriz_rendimiento.rank(axis=1, ascending=False)
    friedman_result = friedmanchisquare(*ranking.T.values)
    ranking_resultado = pd.DataFrame({'Método': ranking.columns, 'Ranking_Promedio': ranking.mean().values})
    ranking_resultado = ranking_resultado.sort_values(by='Ranking_Promedio')
    ranking_resultado.to_csv("ranking_friedman.csv", index=False)

    return df, label_results, resumen_metricas

# -------------------------- RESUMENES Y EXPORTACIONES -------------------------

def resumen_tiempos(df_resultados):
    resumen = df_resultados.dropna(subset=['Tiempo (s)']).groupby(['Representación', 'Método'])['Tiempo (s)'].agg(['mean', 'std']).reset_index()
    resumen.columns = ['Representación', 'Método', 'Tiempo Medio (s)', 'Desviación Std (s)']
    return resumen


In [19]:
# Ejecucion del calculo y obtencion de resultados
# ----------------------------- USO EJECUTABLE FINAL ---------------------------
if __name__ == "__main__":
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)

    df_resultados, etiquetas, resumen_metricas = run_experiments(X)
    resumen = resumen_tiempos(df_resultados)

    df_resultados.to_csv("resultados_validacion_cruzada.csv", index=False)
    resumen.to_csv("resumen_tiempos.csv", index=False)

    params_df = df_resultados[["Representación", "Método", "Params_Representación", "Params_Clustering"]]
    params_df.to_csv("hiperparametros_validacion_cruzada.csv", index=False)

    files.download("resultados_validacion_cruzada.csv")
    files.download("resumen_tiempos.csv")
    files.download("hiperparametros_validacion_cruzada.csv")
    files.download("resumen_metricas_clustering.csv")
    files.download("todos_los_resultados.csv")
    files.download("etiquetas_finales_cross_validation.csv")
    files.download("score_normalizado_por_metodo.csv")
    files.download("ranking_friedman.csv")

    print("✔ Resultados guardados y descargados exitosamente.")

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (4). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:

Error en KernelPCA (kernel=rbf, gamma=0.1) + BIRCH (k=3, t=0.5): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=0.1) + BIRCH (k=3, t=1.0): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=0.1) + BIRCH (k=4, t=0.5): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=0.1) + BIRCH (k=4, t=1.0): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=0.1) + BIRCH (k=5, t=0.5): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=0.1) + BIRCH (k=5, t=1.0): Se generó solo un clúster. No se puede calcular métricas de calidad.


/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (3). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (4). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (4). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/cluster/_birch.py:727: ConvergenceWarning: Number of subclusters found (1) by BIRCH is less than (5). Decrease the threshold.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklea

Error en KernelPCA (kernel=rbf, gamma=1) + BIRCH (k=3, t=0.5): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=1) + BIRCH (k=3, t=1.0): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=1) + BIRCH (k=4, t=0.5): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=1) + BIRCH (k=4, t=1.0): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=1) + BIRCH (k=5, t=0.5): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=rbf, gamma=1) + BIRCH (k=5, t=1.0): Se generó solo un clúster. No se puede calcular métricas de calidad.
Error en KernelPCA (kernel=poly, gamma=0.1) + GMM (k=4, cov=diag): Fitting the mixture model failed because some components have ill-defined empirical covariance (for instance caused by singleton or collapsed

/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:1389: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.11/dist-packages/sklearn/manifold/_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
/usr

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✔ Resultados guardados y descargados exitosamente.
